# Capstone Project - The Battle of Neighbourhoods: Should I move from Lausanne to Bern?
Christopher Tse, Dec 2019
Compare Lausanne and Bern, and using data science techniques determine which neighbourhood would be best for me and explain why.

## Introduction
I am a highly qualified biotech scientist living in Switzerland. I'm currently living in a city called Lausanne, on the French-speaking side of Switzerland, but there may be more job and life opportunities in a German-speaking city called Bern in Switzerland. Bern is very popular among consultancy, biotech and pharmaceutical companies and I want to use data science to help me decide which city would suit me best to live.

I want to compare the livability of the two cities through using similar methods that have been used in the assignments from this course, and help me decide which area would be best for me - a young adult. This project would also be of interest for other individuals individuals and organisations who are interested in assessing these two cities for business and living. 

## Data

In this project, I will use Foursquare data to compare the most interesting regions of Lausanne and Bern. This will be done with k-means to segment and cluster related neighbourhoods to allow me to estimate the suitable demographic to live there. I will be interested in areas which are suitable for a young adult demographic which would include social venues and public transport access for example.
The use of folium interactive maps would be very useful here also so I can visually see attractive features like rivers and parks in the respective areas to strengthen the validity of my recommendations.

There is a limit to how many venues each query can retrieve (up to 113), therefore I will need to break up my query into several queries to cover as much of each city as possible. 
A local governmnet-run supermarket chain called Migros are well distributed across all cities to provide the sale of groceries to the local populations. I will use these as local "neighbourhoods" to capture as many venues as possible. 

Using Foursquare, I first start at the train stations from each city. From the train station, I select all Migros supermarkets nearby. From each Migros supermarket, I list out all nearby venues and analyse each of these "neighbourhoods". 


## Methodology
### Methodology section which represents the main component of the report where you discuss and describe any exploratory data analysis that you did, any inferential statistical testing that you performed, if any, and what machine learnings were used and why.

In [26]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 

print('Libraries imported.')

Solving environment: done

# All requested packages already installed.

Libraries imported.


### Define Foursquare credentials and version

In [27]:
CLIENT_ID = 'OHYIJKDCMRFM0YTYCZRJHC5H2I5ZA51HU41EU5LK1W5PVAPS' # your Foursquare ID
CLIENT_SECRET = 'G2KF03HSYHCLVCHN11ZBGR5GZDVKBYOG2UVF20X4BPQLQ15G' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 50
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: OHYIJKDCMRFM0YTYCZRJHC5H2I5ZA51HU41EU5LK1W5PVAPS
CLIENT_SECRET:G2KF03HSYHCLVCHN11ZBGR5GZDVKBYOG2UVF20X4BPQLQ15G


### For each city, I will start with the train station, and from there, select all nearby Migros supermarkets.

In [28]:
address = '3008 Bern'
# For my work in Bern, I will find '3008 Bern'
# Place de la Gare 5A, 1003 Lausanne
geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)
print(location)

46.9482713 7.4514512
Bern, Verwaltungskreis Bern-Mittelland, Verwaltungsregion Bern-Mittelland, Bern/Berne, Switzerland


### Having the location data, I can now use the Foursquare API to find all the Migros venues around the station. I set a radius of 1000, and previously had set the limit to 50 venues. 

In [29]:
search_query = 'Migros'
radius = 10000
print(search_query + ' .... OK!')

Migros .... OK!


### Define the corresponding URL

In [30]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=OHYIJKDCMRFM0YTYCZRJHC5H2I5ZA51HU41EU5LK1W5PVAPS&client_secret=G2KF03HSYHCLVCHN11ZBGR5GZDVKBYOG2UVF20X4BPQLQ15G&ll=46.9482713,7.4514512&v=20180604&query=Migros&radius=10000&limit=50'

### Send the GET Request and examine the results

In [31]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5df65fad71c428001b7aeb67'},
 'response': {'venues': [{'id': '4b644e06f964a5208ba92ae3',
    'name': 'Migros',
    'location': {'address': 'Marktgasse 46',
     'lat': 46.94813202006709,
     'lng': 7.444727765228771,
     'labeledLatLngs': [{'label': 'display',
       'lat': 46.94813202006709,
       'lng': 7.444727765228771}],
     'distance': 511,
     'postalCode': '3011',
     'cc': 'CH',
     'city': 'Bern',
     'state': 'Bern',
     'country': 'Schweiz',
     'formattedAddress': ['Marktgasse 46', '3011 Bern', 'Schweiz']},
    'categories': [{'id': '52f2ab2ebcbc57f1066b8b46',
      'name': 'Supermarket',
      'pluralName': 'Supermarkets',
      'shortName': 'Supermarket',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/shops/food_grocery_',
       'suffix': '.png'},
      'primary': True}],
    'referralId': 'v-1576427495',
    'hasPerk': False},
   {'id': '4cbdc993dd41a35d2720fba0',
    'name': 'Migros Christoffel',
    'loc

### Get relevant part of JSON and transform it into a *pandas* dataframe

In [32]:
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head(50)

,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.neighborhood,location.postalCode,location.state,name,referralId
0,"[{'id': '52f2ab2ebcbc57f1066b8b46', 'name': 'S...",False,4b644e06f964a5208ba92ae3,Marktgasse 46,CH,Bern,Schweiz,NaN,511,"[Marktgasse 46, 3011 Bern, Schweiz]","[{'label': 'display', 'lat': 46.94813202006709...",46.948132,7.444728,NaN,3011,Bern,Migros,v-1576427495
1,"[{'id': '52f2ab2ebcbc57f1066b8b46', 'name': 'S...",False,4cbdc993dd41a35d2720fba0,Bahnhof Bern,CH,Bern,Schweiz,RailCity Christoffelunterführung,837,[Bahnhof Bern (RailCity Christoffelunterführun...,"[{'label': 'display', 'lat': 46.9475846171591,...",46.947585,7.440479,NaN,3011,Bern,Migros Christoffel,v-1576427495
2,"[{'id': '52f2ab2ebcbc57f1066b8b46', 'name': 'S...",False,4b6f2023f964a520bdde2ce3,Bahnhof Bern,CH,Bern,Schweiz,RailCity Galerie,881,"[Bahnhof Bern (RailCity Galerie), 3011 Bern, S...","[{'label': 'display', 'lat': 46.94925357896667...",46.949254,7.439941,NaN,3011,Bern,Migros Bern-Bahnhof,v-1576427495
3,"[{'id': '52f2ab2ebcbc57f1066b8b46', 'name': 'S...",False,4b5f1cb1f964a520baa629e3,Zähringerstr. 43,CH,Bern,Schweiz,NaN,1536,"[Zähringerstr. 43, 3012 Bern, Schweiz]","[{'label': 'display', 'lat': 46.95521840964324...",46.955218,7.433974,NaN,3012,Bern,Migros,v-1576427495
4,"[{'id': '4bf58dd8d48988d118951735', 'name': 'G...",False,4cbf178d00d837046fbb415c,Thunstr. 18,CH,Bern,Schweiz,NaN,666,"[Thunstr. 18, 3005 Bern, Schweiz]","[{'label': 'display', 'lat': 46.94238893196707...",46.942389,7.453110,NaN,3005,Bern,Migros,v-1576427495
5,"[{'id': '4bf58dd8d48988d1a2941735', 'name': 'C...",False,4da717fe0cb66f658712fbb3,Marktgasse 46,CH,Bern,Schweiz,NaN,496,"[Marktgasse 46, 3011 Bern, Schweiz]","[{'label': 'display', 'lat': 46.94813499952044...",46.948135,7.444923,NaN,3011,Bern,Klubschule Migros,v-1576427495
6,"[{'id': '4bf58dd8d48988d118951735', 'name': 'G...",False,4b7acda1f964a520923d2fe3,Bubenbergplatz 8,CH,Bern,Schweiz,NaN,1043,"[Bubenbergplatz 8, 3011 Bern, Schweiz]","[{'label': 'display', 'lat': 46.94729427209867...",46.947294,7.437797,NaN,3011,Bern,Migros,v-1576427495
7,"[{'id': '4bf58dd8d48988d1fd941735', 'name': 'S...",False,4eda35064901c8be1d4a50a0,Wankdorffeldstr. 71,CH,Bern,Schweiz,Wyler,2113,"[Wankdorffeldstr. 71 (Wyler), 3014 Bern, Schweiz]","[{'label': 'display', 'lat': 46.96546718575483...",46.965467,7.463245,NaN,3014,Bern,Migros,v-1576427495
8,"[{'id': '4bf58dd8d48988d118951735', 'name': 'G...",False,4d6923161b63a1cdb6933b2d,Seftigenstr. 1,CH,Bern,Schweiz,Eigerplatz,1749,"[Seftigenstr. 1 (Eigerplatz), 3007 Bern, Schweiz]","[{'label': 'display', 'lat': 46.94017070967135...",46.940171,7.431724,NaN,3007,Bern,Migros,v-1576427495
9,"[{'id': '4bf58dd8d48988d118951735', 'name': 'G...",False,4b7cee3cf964a5208ca92fe3,Egghölzlistr. 1a,CH,Bern,Schweiz,NaN,2505,"[Egghölzlistr. 1a, 3006 Bern, Schweiz]","[{'label': 'display', 'lat': 46.93509110788873...",46.935091,7.478171,NaN,3006,Bern,Migros,v-1576427495


### Define information of interest and filter dataframe

In [33]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,neighborhood,postalCode,state,id
0,Migros,Supermarket,Marktgasse 46,CH,Bern,Schweiz,NaN,511,"[Marktgasse 46, 3011 Bern, Schweiz]","[{'label': 'display', 'lat': 46.94813202006709...",46.948132,7.444728,NaN,3011,Bern,4b644e06f964a5208ba92ae3
1,Migros Christoffel,Supermarket,Bahnhof Bern,CH,Bern,Schweiz,RailCity Christoffelunterführung,837,[Bahnhof Bern (RailCity Christoffelunterführun...,"[{'label': 'display', 'lat': 46.9475846171591,...",46.947585,7.440479,NaN,3011,Bern,4cbdc993dd41a35d2720fba0
2,Migros Bern-Bahnhof,Supermarket,Bahnhof Bern,CH,Bern,Schweiz,RailCity Galerie,881,"[Bahnhof Bern (RailCity Galerie), 3011 Bern, S...","[{'label': 'display', 'lat': 46.94925357896667...",46.949254,7.439941,NaN,3011,Bern,4b6f2023f964a520bdde2ce3
3,Migros,Supermarket,Zähringerstr. 43,CH,Bern,Schweiz,NaN,1536,"[Zähringerstr. 43, 3012 Bern, Schweiz]","[{'label': 'display', 'lat': 46.95521840964324...",46.955218,7.433974,NaN,3012,Bern,4b5f1cb1f964a520baa629e3
4,Migros,Grocery Store,Thunstr. 18,CH,Bern,Schweiz,NaN,666,"[Thunstr. 18, 3005 Bern, Schweiz]","[{'label': 'display', 'lat': 46.94238893196707...",46.942389,7.453110,NaN,3005,Bern,4cbf178d00d837046fbb415c
5,Klubschule Migros,Community College,Marktgasse 46,CH,Bern,Schweiz,NaN,496,"[Marktgasse 46, 3011 Bern, Schweiz]","[{'label': 'display', 'lat': 46.94813499952044...",46.948135,7.444923,NaN,3011,Bern,4da717fe0cb66f658712fbb3
6,Migros,Grocery Store,Bubenbergplatz 8,CH,Bern,Schweiz,NaN,1043,"[Bubenbergplatz 8, 3011 Bern, Schweiz]","[{'label': 'display', 'lat': 46.94729427209867...",46.947294,7.437797,NaN,3011,Bern,4b7acda1f964a520923d2fe3
7,Migros,Shopping Mall,Wankdorffeldstr. 71,CH,Bern,Schweiz,Wyler,2113,"[Wankdorffeldstr. 71 (Wyler), 3014 Bern, Schweiz]","[{'label': 'display', 'lat': 46.96546718575483...",46.965467,7.463245,NaN,3014,Bern,4eda35064901c8be1d4a50a0
8,Migros,Grocery Store,Seftigenstr. 1,CH,Bern,Schweiz,Eigerplatz,1749,"[Seftigenstr. 1 (Eigerplatz), 3007 Bern, Schweiz]","[{'label': 'display', 'lat': 46.94017070967135...",46.940171,7.431724,NaN,3007,Bern,4d6923161b63a1cdb6933b2d
9,Migros,Grocery Store,Egghölzlistr. 1a,CH,Bern,Schweiz,NaN,2505,"[Egghölzlistr. 1a, 3006 Bern, Schweiz]","[{'label': 'display', 'lat': 46.93509110788873...",46.935091,7.478171,NaN,3006,Bern,4b7cee3cf964a5208ca92fe3


### Let's visualize the venues that are in Lausanne, with a focus from the train station

In [34]:
dataframe_filtered.name

0                          Migros
1              Migros Christoffel
2             Migros Bern-Bahnhof
3                          Migros
4                          Migros
5               Klubschule Migros
6                          Migros
7                          Migros
8                          Migros
9                          Migros
10                         Migros
11                         Migros
12              Klubschule Migros
13              Klubschule Migros
14                     Migros VOI
15          Migros Do It + Garden
16                         Migros
17          Do It + Garden Migros
18              Migros Restaurant
19                         Migros
20                   Migros Daily
21                         Migros
22               Migros Take Away
23                         Migros
24                         Migros
25                     Migros VOI
26               Migros Take Away
27                    Migros Bank
28                         Migros
29            

In [35]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=13) # generate map centred around the Conrad Hotel

# add a red circle marker to represent the train sation
folium.features.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='red',
    popup='Lausanne Gare',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map)

# add the venues as blue circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)

# display map
venues_map

### Let's create a function to repeat the same process to all the areas near a Mirgos in Lausanne

In [36]:
#
def getNearbyVenues(names, latitudes, longitudes, radius=1000):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

### Now we write the code to run the above function on each neighborhood and create a new dataframe called *lausanne_areas*, with the unique ID of each Migros store

In [37]:
lausanne_areas = getNearbyVenues(names=dataframe_filtered['id'],
                                   latitudes=dataframe_filtered['lat'],
                                   longitudes=dataframe_filtered['lng']
                                  )

4b644e06f964a5208ba92ae3
4cbdc993dd41a35d2720fba0
4b6f2023f964a520bdde2ce3
4b5f1cb1f964a520baa629e3
4cbf178d00d837046fbb415c
4da717fe0cb66f658712fbb3
4b7acda1f964a520923d2fe3
4eda35064901c8be1d4a50a0
4d6923161b63a1cdb6933b2d
4b7cee3cf964a5208ca92fe3
4b75b135f964a520661d2ee3
4b51ead6f964a520385b27e3
4cc930035c17b60c9ea30705
4d7665b888235481ad046a8c
53ec9a0b498e318087cc0186
4bd046df046076b0a69d6f71
52dcd06311d23f81f2998b26
4bf564cbff90c9b69d4d5628
4d81eb80c8bdb1f7a5aed2c2
530de9db498e5909d3f5ba68
59def3e0e55d8b6365bb9ed9
4cbff5b0020d468836756538
4b7a85a3f964a520bb2f2fe3
4d94b1b33048b1f7af213764
4c74e2953adda143afbf05af
4eaed0bc7ee59f16098555fd
5422a3de498ee9964a114d75
52fb94fe498e1ff912412882
53272ee9498ec7c75fe39d63
50cf0feae4b0c490cfcc597d
4d13025f2e58370486d2e8d1
4cf79ed0e308236a2b63ecac
57778ed838fad7362ffc78a9
4bee662ce8c3c9288ea89892
4d8ddaa1788c5481e32b2afd
4b58503cf964a520425228e3
4dcbef3052b17cba4fb31068
4ec65d8a0e6158bafd288ac4
56027f5b498e5def6f49523d
4d9459ae98c88cfac1ac1949


### Let's check the size of the resulting dataframe

In [38]:
print(lausanne_areas.shape)
lausanne_areas.head()

(1612, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,4b644e06f964a5208ba92ae3,46.948132,7.444728,Bundesplatz,46.947414,7.443817,Plaza
1,4b644e06f964a5208ba92ae3,46.948132,7.444728,Adriano's Bar & Café,46.947914,7.447428,Café
2,4b644e06f964a5208ba92ae3,46.948132,7.444728,Zytglogge,46.948084,7.447555,Monument / Landmark
3,4b644e06f964a5208ba92ae3,46.948132,7.444728,Bellevue Palace Bern,46.946799,7.446866,Hotel
4,4b644e06f964a5208ba92ae3,46.948132,7.444728,Migros,46.948132,7.444728,Supermarket


In [39]:
print('There are {} uniques categories.'.format(len(lausanne_areas['Venue Category'].unique())))

There are 131 uniques categories.


In [40]:
# one hot encoding
lausanne_onehot = pd.get_dummies(lausanne_areas[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
lausanne_onehot['Neighborhood'] = lausanne_areas['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [lausanne_onehot.columns[-1]] + list(lausanne_onehot.columns[:-1])
lausanne_onehot = lausanne_onehot[fixed_columns]

lausanne_onehot.head()

,Zoo Exhibit,Art Gallery,Art Museum,Asian Restaurant,Auto Garage,Bagel Shop,Bakery,Bar,Bathing Area,Beach,Beer Garden,Bistro,Bookstore,Bowling Alley,Breakfast Spot,Brewery,Buffet,Burger Joint,Bus Station,Bus Stop,Café,Campground,Chinese Restaurant,Church,Clothing Store,Cocktail Bar,Coffee Shop,Concert Hall,Construction & Landscaping,Convenience Store,Convention Center,Coworking Space,Creperie,Cupcake Shop,Department Store,Dessert Shop,Discount Store,Electronics Store,Event Space,Fast Food Restaurant,Flower Shop,Food,Food & Drink Shop,Food Truck,Forest,French Restaurant,Frozen Yogurt Shop,Furniture / Home Store,Garden,Gas Station,Gastropub,General Entertainment,Gourmet Shop,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hardware Store,Historic Site,History Museum,Hockey Arena,Hockey Field,Home Service,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Israeli Restaurant,Italian Restaurant,Jazz Club,Lake,Laundry Service,Light Rail Station,Liquor Store,Massage Studio,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Modern European Restaurant,Monument / Landmark,Multiplex,Music Venue,Neighborhood,Nightclub,Office,Paper / Office Supplies Store,Park,Performing Arts Venue,Pharmacy,Pizza Place,Plaza,Pool,Ramen Restaurant,Rental Car Location,Restaurant,Rock Club,Sandwich Place,Scenic Lookout,Science Museum,Shoe Store,Shopping Mall,Skating Rink,Snack Place,Soccer Field,Soccer Stadium,Spa,Spanish Restaurant,Sporting Goods Shop,Sports Bar,Stadium,Steakhouse,Storage Facility,Supermarket,Sushi Restaurant,Swiss Restaurant,Tapas Restaurant,Tennis Court,Thai Restaurant,Theater,Track,Train Station,Tram Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Volleyball Court,Water Park,Wine Bar,Zoo
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4b644e06f964a5208ba92ae3,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4b644e06f964a5208ba92ae3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,4b644e06f964a5208ba92ae3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4b644e06f964a5208ba92ae3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4b644e06f964a5208ba92ae3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


### Let's check how many venues were returned for each neighborhood

In [41]:
lausanne_onehot.shape

(1612, 131)

In [42]:
lausanne_grouped = lausanne_onehot.groupby('Neighborhood').mean().reset_index()
lausanne_grouped

,Neighborhood,Zoo Exhibit,Art Gallery,Art Museum,Asian Restaurant,Auto Garage,Bagel Shop,Bakery,Bar,Bathing Area,Beach,Beer Garden,Bistro,Bookstore,Bowling Alley,Breakfast Spot,Brewery,Buffet,Burger Joint,Bus Station,Bus Stop,Café,Campground,Chinese Restaurant,Church,Clothing Store,Cocktail Bar,Coffee Shop,Concert Hall,Construction & Landscaping,Convenience Store,Convention Center,Coworking Space,Creperie,Cupcake Shop,Department Store,Dessert Shop,Discount Store,Electronics Store,Event Space,Fast Food Restaurant,Flower Shop,Food,Food & Drink Shop,Food Truck,Forest,French Restaurant,Frozen Yogurt Shop,Furniture / Home Store,Garden,Gas Station,Gastropub,General Entertainment,Gourmet Shop,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hardware Store,Historic Site,History Museum,Hockey Arena,Hockey Field,Home Service,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Israeli Restaurant,Italian Restaurant,Jazz Club,Lake,Laundry Service,Light Rail Station,Liquor Store,Massage Studio,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Modern European Restaurant,Monument / Landmark,Multiplex,Music Venue,Nightclub,Office,Paper / Office Supplies Store,Park,Performing Arts Venue,Pharmacy,Pizza Place,Plaza,Pool,Ramen Restaurant,Rental Car Location,Restaurant,Rock Club,Sandwich Place,Scenic Lookout,Science Museum,Shoe Store,Shopping Mall,Skating Rink,Snack Place,Soccer Field,Soccer Stadium,Spa,Spanish Restaurant,Sporting Goods Shop,Sports Bar,Stadium,Steakhouse,Storage Facility,Supermarket,Sushi Restaurant,Swiss Restaurant,Tapas Restaurant,Tennis Court,Thai Restaurant,Theater,Track,Train Station,Tram Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Volleyball Court,Water Park,Wine Bar,Zoo
0,4b51ead6f964a520385b27e3,0.000000,0.00,0.000000,0.083333,0.00,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.00,0.00,0.00000,0.000000,0.00,0.166667,0.083333,0.00,0.00,0.00,0.00,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.00,0.00,0.00,0.00,0.083333,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.083333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.083333,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.00000,0.000000,0.00,0.00,0.000000,0.000000,0.00,0.000000,0.00,0.00,0.00000,0.000000,0.00,0.0,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.00,0.00,0.000000,0.000000,0.083333,0.00000,0.000000,0.000000,0.000000,0.083333,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.083333,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.166667,0.000000,0.000000,0.00,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.00
1,4b58503cf964a520425228e3,0.000000,0.00,0.000000,0.047619,0.00,0.00,0.047619,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.00,0.00,0.00000,0.000000,0.00,0.000000,0.000000,0.00,0.00,0.00,0.00,0.000000,0.00,0.047619,0.00,0.000000,0.000000,0.00,0.00,0.00,0.00,0.047619,0.00,0.000000,0.000000,0.00,0.095238,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.047619,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.095238,0.000000,0.047619,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.00000,0.000000,0.00,0.00,0.000000,0.000000,0.00,0.047619,0.00,0.00,0.00000,0.047619,0.00,0.0,0.000000,0.047619,0.00,0.000000,0.000000,0.00,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.047619,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.047619,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.142857,0.047619,0.047619,0.00,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.047619,0.00,0.00
2,4b5f1cb1f964a520baa629e3,0.000000,0.00,0.020000,0.020000,0.00,0.02,0.000000,0.140000,0.00,0.000000,0.000000,0.000000,0.02,0.00,0.00,0.00000,0.000000,0.02,0.000000,0.000000,0.08,0.00,0.00,0.00,0.020000,0.00,0.

### Let's print each neighborhood along with the top 5 most common venues

In [43]:
num_top_venues = 5

for hood in lausanne_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = lausanne_grouped[lausanne_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----4b51ead6f964a520385b27e3----
           venue  freq
0    Bus Station  0.17
1    Supermarket  0.17
2  Grocery Store  0.08
3  Shopping Mall  0.08
4           Pool  0.08


----4b58503cf964a520425228e3----
                  venue  freq
0           Supermarket  0.14
1         Grocery Store  0.10
2  Fast Food Restaurant  0.10
3      Swiss Restaurant  0.05
4      Ramen Restaurant  0.05


----4b5f1cb1f964a520baa629e3----
                venue  freq
0                 Bar  0.14
1          Restaurant  0.10
2                Café  0.08
3  Italian Restaurant  0.08
4                Park  0.04


----4b644e06f964a5208ba92ae3----
                venue  freq
0                Café  0.10
1               Plaza  0.08
2  Italian Restaurant  0.08
3                 Bar  0.06
4                Park  0.06


----4b6f2023f964a520bdde2ce3----
                venue  freq
0  Italian Restaurant  0.08
1               Plaza  0.08
2                Café  0.06
3                Park  0.06
4          Restaurant  0.06


---

### Let's put that into a *pandas* dataframe

In [44]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [45]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = lausanne_grouped['Neighborhood']

for ind in np.arange(lausanne_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(lausanne_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,4b51ead6f964a520385b27e3,Supermarket,Bus Station,Pool,Park,Food Truck,Grocery Store,Shopping Mall,Asian Restaurant,Department Store,Bus Stop
1,4b58503cf964a520425228e3,Supermarket,Fast Food Restaurant,Grocery Store,Light Rail Station,Coffee Shop,Mexican Restaurant,Frozen Yogurt Shop,Shopping Mall,Sushi Restaurant,Swiss Restaurant
2,4b5f1cb1f964a520baa629e3,Bar,Restaurant,Café,Italian Restaurant,Creperie,Park,Electronics Store,Event Space,Pizza Place,Israeli Restaurant
3,4b644e06f964a5208ba92ae3,Café,Italian Restaurant,Plaza,Bar,Park,Swiss Restaurant,Hotel,Restaurant,Burger Joint,Food & Drink Shop
4,4b6f2023f964a520bdde2ce3,Plaza,Italian Restaurant,Café,Park,Restaurant,Monument / Landmark,Swiss Restaurant,Creperie,Hotel,Food & Drink Shop


## Cluster Neighborhoods
Now I check the similarities of each neighbourhood with one another using k clusters 

In [46]:
# set number of clusters
kclusters = 8

lausanne_grouped_clustering = lausanne_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(lausanne_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([7, 5, 0, 0, 0, 3, 0, 0, 3, 3], dtype=int32)

In [47]:
neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,4b51ead6f964a520385b27e3,Supermarket,Bus Station,Pool,Park,Food Truck,Grocery Store,Shopping Mall,Asian Restaurant,Department Store,Bus Stop
1,4b58503cf964a520425228e3,Supermarket,Fast Food Restaurant,Grocery Store,Light Rail Station,Coffee Shop,Mexican Restaurant,Frozen Yogurt Shop,Shopping Mall,Sushi Restaurant,Swiss Restaurant
2,4b5f1cb1f964a520baa629e3,Bar,Restaurant,Café,Italian Restaurant,Creperie,Park,Electronics Store,Event Space,Pizza Place,Israeli Restaurant
3,4b644e06f964a5208ba92ae3,Café,Italian Restaurant,Plaza,Bar,Park,Swiss Restaurant,Hotel,Restaurant,Burger Joint,Food & Drink Shop
4,4b6f2023f964a520bdde2ce3,Plaza,Italian Restaurant,Café,Park,Restaurant,Monument / Landmark,Swiss Restaurant,Creperie,Hotel,Food & Drink Shop
5,4b75b135f964a520661d2ee3,Supermarket,Restaurant,Discount Store,Soccer Field,Swiss Restaurant,Bistro,Bus Station,Paper / Office Supplies Store,Office,Grocery Store
6,4b7a85a3f964a520bb2f2fe3,Plaza,Italian Restaurant,Café,Park,Restaurant,Monument / Landmark,Swiss Restaurant,Creperie,Hotel,Food & Drink Shop
7,4b7acda1f964a520923d2fe3,Café,Plaza,Creperie,Park,Restaurant,Vegetarian / Vegan Restaurant,Pizza Place,Bar,Hotel,Coffee Shop
8,4b7cee3cf964a5208ca92fe3,Tram Station,Supermarket,Hotel,Food & Drink Shop,Light Rail Station,Beach,Grocery Store,Restaurant,Bus Stop,Electronics Store
9,4bd046df046076b0a69d6f71,Italian Restaurant,Swiss Restaurant,Hotel,Bus Station,Restaurant,Stadium,Supermarket,Shopping Mall,Tram Station,Laundry Service


In [48]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

lausanne_merged = dataframe_filtered

# merge lausanne_grouped with lausanne_data to add latitude/longitude for each neighborhood
lausanne_merged = lausanne_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='id')
#dataframe_filtered
lausanne_merged

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,neighborhood,postalCode,state,id,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Migros,Supermarket,Marktgasse 46,CH,Bern,Schweiz,NaN,511,"[Marktgasse 46, 3011 Bern, Schweiz]","[{'label': 'display', 'lat': 46.94813202006709...",46.948132,7.444728,NaN,3011,Bern,4b644e06f964a5208ba92ae3,0,Café,Italian Restaurant,Plaza,Bar,Park,Swiss Restaurant,Hotel,Restaurant,Burger Joint,Food & Drink Shop
1,Migros Christoffel,Supermarket,Bahnhof Bern,CH,Bern,Schweiz,RailCity Christoffelunterführung,837,[Bahnhof Bern (RailCity Christoffelunterführun...,"[{'label': 'display', 'lat': 46.9475846171591,...",46.947585,7.440479,NaN,3011,Bern,4cbdc993dd41a35d2720fba0,0,Café,Italian Restaurant,Plaza,Park,Swiss Restaurant,Monument / Landmark,Pizza Place,Hotel,Restaurant,Liquor Store
2,Migros Bern-Bahnhof,Supermarket,Bahnhof Bern,CH,Bern,Schweiz,RailCity Galerie,881,"[Bahnhof Bern (RailCity Galerie), 3011 Bern, S...","[{'label': 'display', 'lat': 46.94925357896667...",46.949254,7.439941,NaN,3011,Bern,4b6f2023f964a520bdde2ce3,0,Plaza,Italian Restaurant,Café,Park,Restaurant,Monument / Landmark,Swiss Restaurant,Creperie,Hotel,Food & Drink Shop
3,Migros,Supermarket,Zähringerstr. 43,CH,Bern,Schweiz,NaN,1536,"[Zähringerstr. 43, 3012 Bern, Schweiz]","[{'label': 'display', 'lat': 46.95521840964324...",46.955218,7.433974,NaN,3012,Bern,4b5f1cb1f964a520baa629e3,0,Bar,Restaurant,Café,Italian Restaurant,Creperie,Park,Electronics Store,Event Space,Pizza Place,Israeli Restaurant
4,Migros,Grocery Store,Thunstr. 18,CH,Bern,Schweiz,NaN,666,"[Thunstr. 18, 3005 Bern, Schweiz]","[{'label': 'display', 'lat': 46.94238893196707...",46.942389,7.453110,NaN,3005,Bern,4cbf178d00d837046fbb415c,0,Swiss Restaurant,Plaza,Bar,Café,Park,Science Museum,Zoo,Wine Bar,Pizza Place,Performing Arts Venue
5,Klubschule Migros,Community College,Marktgasse 46,CH,Bern,Schweiz,NaN,496,"[Marktgasse 46, 3011 Bern, Schweiz]","[{'label': 'display', 'lat': 46.94813499952044...",46.948135,7.444923,NaN,3011,Bern,4da717fe0cb66f658712fbb3,0,Café,Italian Restaurant,Plaza,Bar,Park,Hotel,Restaurant,Swiss Restaurant,Nightclub,Clothing Store
6,Migros,Grocery Store,Bubenbergplatz 8,CH,Bern,Schweiz,NaN,1043,"[Bubenbergplatz 8, 3011 Bern, Schweiz]","[{'label': 'display', 'lat': 46.94729427209867...",46.947294,7.437797,NaN,3011,Bern,4b7acda1f964a520923d2fe3,0,Café,Plaza,Creperie,Park,Restaurant,Vegetarian / Vegan Restaurant,Pizza Place,Bar,Hotel,Coffee Shop
7,Migros,Shopping Mall,Wankdorffeldstr. 71,CH,Bern,Schweiz,Wyler,2113,"[Wankdorffeldstr. 71 (Wyler), 3014 Bern, Schweiz]","[{'label': 'display', 'lat': 46.96546718575483...",46.965467,7.463245,NaN,3014,Bern,4eda35064901c8be1d4a50a0,3,Italian Restaurant,Restaurant,Swiss Restaurant,Supermarket,Bus Station,Shopping Mall,Stadium,Tram Station,Hotel,Clothing Store
8,Migros,Grocery Store,Seftigenstr. 1,CH,Bern,Schweiz,Eigerplatz,1749,"[Seftigenstr. 1 (Eigerplatz), 3007 Bern, Schweiz]","[{'label': 'display', 'lat': 46.94017070967135...",46.940171,7.431724,NaN,3007,Bern,4d6923161b63a1cdb6933b2d,3,Tram Station,Pizza Place,Supermarket,Swiss Restaurant,Plaza,Hotel,Bakery,Italian Restaurant,Park,Cupcake Shop
9,Migros,Grocery Store,Egghölzlistr. 1a,CH,Bern,Schweiz,NaN,2505,"[Egghölzlistr. 1a, 3006 Bern, Schweiz]","[{'label': 'display', 'lat': 46.93509110788873...",46.935091,7.478171,NaN,3006,Bern,4b7cee3cf964a5208ca92fe3,3,Tram Station,Supermarket,Hotel,Food & Drink Shop,Light Rail Station,Beach,Grocery Store,Restaurant,Bus Stop,Electronics Store


In [49]:
lausanne_merged

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,neighborhood,postalCode,state,id,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Migros,Supermarket,Marktgasse 46,CH,Bern,Schweiz,NaN,511,"[Marktgasse 46, 3011 Bern, Schweiz]","[{'label': 'display', 'lat': 46.94813202006709...",46.948132,7.444728,NaN,3011,Bern,4b644e06f964a5208ba92ae3,0,Café,Italian Restaurant,Plaza,Bar,Park,Swiss Restaurant,Hotel,Restaurant,Burger Joint,Food & Drink Shop
1,Migros Christoffel,Supermarket,Bahnhof Bern,CH,Bern,Schweiz,RailCity Christoffelunterführung,837,[Bahnhof Bern (RailCity Christoffelunterführun...,"[{'label': 'display', 'lat': 46.9475846171591,...",46.947585,7.440479,NaN,3011,Bern,4cbdc993dd41a35d2720fba0,0,Café,Italian Restaurant,Plaza,Park,Swiss Restaurant,Monument / Landmark,Pizza Place,Hotel,Restaurant,Liquor Store
2,Migros Bern-Bahnhof,Supermarket,Bahnhof Bern,CH,Bern,Schweiz,RailCity Galerie,881,"[Bahnhof Bern (RailCity Galerie), 3011 Bern, S...","[{'label': 'display', 'lat': 46.94925357896667...",46.949254,7.439941,NaN,3011,Bern,4b6f2023f964a520bdde2ce3,0,Plaza,Italian Restaurant,Café,Park,Restaurant,Monument / Landmark,Swiss Restaurant,Creperie,Hotel,Food & Drink Shop
3,Migros,Supermarket,Zähringerstr. 43,CH,Bern,Schweiz,NaN,1536,"[Zähringerstr. 43, 3012 Bern, Schweiz]","[{'label': 'display', 'lat': 46.95521840964324...",46.955218,7.433974,NaN,3012,Bern,4b5f1cb1f964a520baa629e3,0,Bar,Restaurant,Café,Italian Restaurant,Creperie,Park,Electronics Store,Event Space,Pizza Place,Israeli Restaurant
4,Migros,Grocery Store,Thunstr. 18,CH,Bern,Schweiz,NaN,666,"[Thunstr. 18, 3005 Bern, Schweiz]","[{'label': 'display', 'lat': 46.94238893196707...",46.942389,7.453110,NaN,3005,Bern,4cbf178d00d837046fbb415c,0,Swiss Restaurant,Plaza,Bar,Café,Park,Science Museum,Zoo,Wine Bar,Pizza Place,Performing Arts Venue
5,Klubschule Migros,Community College,Marktgasse 46,CH,Bern,Schweiz,NaN,496,"[Marktgasse 46, 3011 Bern, Schweiz]","[{'label': 'display', 'lat': 46.94813499952044...",46.948135,7.444923,NaN,3011,Bern,4da717fe0cb66f658712fbb3,0,Café,Italian Restaurant,Plaza,Bar,Park,Hotel,Restaurant,Swiss Restaurant,Nightclub,Clothing Store
6,Migros,Grocery Store,Bubenbergplatz 8,CH,Bern,Schweiz,NaN,1043,"[Bubenbergplatz 8, 3011 Bern, Schweiz]","[{'label': 'display', 'lat': 46.94729427209867...",46.947294,7.437797,NaN,3011,Bern,4b7acda1f964a520923d2fe3,0,Café,Plaza,Creperie,Park,Restaurant,Vegetarian / Vegan Restaurant,Pizza Place,Bar,Hotel,Coffee Shop
7,Migros,Shopping Mall,Wankdorffeldstr. 71,CH,Bern,Schweiz,Wyler,2113,"[Wankdorffeldstr. 71 (Wyler), 3014 Bern, Schweiz]","[{'label': 'display', 'lat': 46.96546718575483...",46.965467,7.463245,NaN,3014,Bern,4eda35064901c8be1d4a50a0,3,Italian Restaurant,Restaurant,Swiss Restaurant,Supermarket,Bus Station,Shopping Mall,Stadium,Tram Station,Hotel,Clothing Store
8,Migros,Grocery Store,Seftigenstr. 1,CH,Bern,Schweiz,Eigerplatz,1749,"[Seftigenstr. 1 (Eigerplatz), 3007 Bern, Schweiz]","[{'label': 'display', 'lat': 46.94017070967135...",46.940171,7.431724,NaN,3007,Bern,4d6923161b63a1cdb6933b2d,3,Tram Station,Pizza Place,Supermarket,Swiss Restaurant,Plaza,Hotel,Bakery,Italian Restaurant,Park,Cupcake Shop
9,Migros,Grocery Store,Egghölzlistr. 1a,CH,Bern,Schweiz,NaN,2505,"[Egghölzlistr. 1a, 3006 Bern, Schweiz]","[{'label': 'display', 'lat': 46.93509110788873...",46.935091,7.478171,NaN,3006,Bern,4b7cee3cf964a5208ca92fe3,3,Tram Station,Supermarket,Hotel,Food & Drink Shop,Light Rail Station,Beach,Grocery Store,Restaurant,Bus Stop,Electronics Store


In [50]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(lausanne_merged['lat'], lausanne_merged['lng'], lausanne_merged['id'], lausanne_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

# Bern next!
##The exact same code was used for Bern, just the starting point was changed to the Bern train station. It would have been a waste of time to copy and paste the same code, so I recycled the prvious code used for Lausanne to find the best places in Bern. My presentation will show this and the resiults. 

## Results
Results section where you discuss the results.

## Discussion
Discussion section where you discuss any observations you noted and any recommendations you can make based on the results.

## Conclusion
Conclusion section where you conclude the report.